<a href="https://www.kaggle.com/code/nigamshitij/nlp-tokenizer-classifier-huggingface-tfmr?scriptVersionId=177048331" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv
/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv


In [3]:
import matplotlib.pyplot as pl
import pandas as pd
import torch
from fastai.imports import *

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
iskaggle

'Interactive'

In [4]:
if iskaggle:
    path = Path('/kaggle/input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [5]:
# !ls {path}
dataframe = pd.read_csv(path/'train.csv')
dataframe.rename(columns = {'score': 'label'}, inplace = True)
print(dataframe.head())
# dataframe.describe(include = 'object')

dataframe['input'] = 'TEXT1: ' + dataframe.context + '; TEXT2: ' + dataframe.target + '; ANC1: ' + dataframe.anchor
# dataframe['input'].head()
dataframe.describe(include = 'object')

                 id     anchor                  target context  label
0  37d61fd2272659b1  abatement  abatement of pollution     A47   0.50
1  7b9652b17b68b7a4  abatement          act of abating     A47   0.75
2  36d72442aefd8232  abatement         active catalyst     A47   0.25
3  5296b0c19e1ce60e  abatement     eliminating process     A47   0.50
4  54c1e3b9184cb5b6  abatement           forest region     A47   0.00


,id,anchor,target,context,input
count,36473,36473,36473,36473,36473
unique,36473,733,29340,106,36473
top,8d135da0b55b8c88,component composite coating,composition,H01,TEXT1: B44; TEXT2: wooden substrate; ANC1: wood article
freq,1,152,24,2186,1


In [6]:
# tokenization
from datasets import Dataset, DatasetDict
dataset = Dataset.from_pandas(dataframe)
dataset

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'label', 'input'],
    num_rows: 36473
})

In [7]:
model_name = 'microsoft/deberta-v3-small'
# https://huggingface.co/microsoft/deberta-v3-small

from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
tokenizer.tokenize('Hello my name is Shitij')

['▁Hello', '▁my', '▁name', '▁is', '▁Shit', 'ij']

In [9]:
# numericalization

def tokenization_function(x):
    return tokenizer(x["input"])

tokenization_dataset = dataset.map(tokenization_function, batched = True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [10]:
row = tokenization_dataset[33000]
row['input'], row['input_ids']

('TEXT1: G21; TEXT2: therapeutic dose; ANC1: therapeutic applications',
 [1,
  54453,
  435,
  294,
  1098,
  2917,
  346,
  54453,
  445,
  294,
  8068,
  5121,
  346,
  23702,
  435,
  294,
  8068,
  1567,
  2])

In [11]:
# tokenizer.tokenize('therapeutic')
# tokenizer.vocab['therap']
# tokenizer.get_vocab()['_therapeutic']

eval_dataframe = pd.read_csv(path/'test.csv')
eval_dataframe.rename(columns = {'score': 'label'}, inplace = True)
eval_dataframe.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [12]:
dataset_dict = tokenization_dataset.train_test_split(0.25, seed = 42)
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'label', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'label', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [13]:
# define correlation coefficient
def corr(x, y):
    return np.corrcoef(x, y)[0][1]

def corr_dictionary(eval_pred):
    return {'pearson': corr(*eval_pred)}

In [14]:
from transformers import TrainingArguments, Trainer

batch_size = 128 # larger, more in parallel, out of memory if too big
epochs = 4
learning_rate = 8e-5

2024-05-11 13:46:04.808486: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 13:46:04.808586: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 13:46:05.078428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [15]:
arguments = TrainingArguments(output_dir = 'outputs',
                              learning_rate = learning_rate,
                              warmup_ratio = 0.1,
                              lr_scheduler_type = 'cosine',
#                               fp16 = True,
                              evaluation_strategy = 'epoch',
                              per_device_train_batch_size = batch_size,
                              per_device_eval_batch_size = batch_size*2,
                              num_train_epochs = epochs,
                              weight_decay = 0.01,
                              report_to = 'none'
                             )
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 1)
trainer = Trainer(model,
                  arguments,
                  train_dataset = dataset_dict['train'],
                  eval_dataset = dataset_dict['test'],
                  tokenizer = tokenizer,
                  compute_metrics = corr_dictionary
                 )

pytorch_model.bin:   0%|          | 0.00/286M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batc

In [16]:
# TrainingArguments??
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.030553,0.788216
2,No log,0.025993,0.817216
3,No log,0.025985,0.825833
4,No log,0.024476,0.827091


TrainOutput(global_step=428, training_loss=0.03380305967598318, metrics={'train_runtime': 270.6332, 'train_samples_per_second': 404.296, 'train_steps_per_second': 1.581, 'total_flos': 748113626405160.0, 'train_loss': 0.03380305967598318, 'epoch': 4.0})

In [17]:
# add input field for tokenization
eval_dataframe['input'] = 'TEXT1: ' + eval_dataframe.context + '; TEXT2: ' + eval_dataframe.target + '; ANC1: ' + eval_dataframe.anchor

# convert dataframe to dataset
eval_dataset = Dataset.from_pandas(eval_dataframe)

# map dataset to tokens using tokenization function
eval_dataset = eval_dataset.map(tokenization_function, batched = True)

# run predictions on dataset using trained model (needs to have input fields)
predictions = trainer.predict(eval_dataset).predictions.astype(float)

# print predictions
predictions

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

array([[ 0.55404937],
       [ 0.71638495],
       [ 0.51819497],
       [ 0.36538407],
       [-0.02204065],
       [ 0.51772523],
       [ 0.48330551],
       [ 0.10196796],
       [ 0.2809881 ],
       [ 1.10844219],
       [ 0.30006972],
       [ 0.31430319],
       [ 0.80609947],
       [ 0.82189167],
       [ 0.79629403],
       [ 0.3811202 ],
       [ 0.31546891],
       [ 0.00935643],
       [ 0.64860564],
       [ 0.37407187],
       [ 0.47903419],
       [ 0.25257939],
       [ 0.11335076],
       [ 0.24104178],
       [ 0.59749633],
       [-0.03576969],
       [-0.04324893],
       [-0.02253797],
       [-0.05716192],
       [ 0.66999573],
       [ 0.35355464],
       [ 0.06568331],
       [ 0.71808344],
       [ 0.46307623],
       [ 0.47121704],
       [ 0.19375861]])